## Gantry

**A Python DICOM Object Model and Redaction Toolkit.**

Gantry provides a high-performance, object-oriented interface for managing, analyzing, and de-identifying DICOM datasets. It is designed for large-scale ingestion, precise pixel redaction, and strict PHI compliance.

1. **Ingest**: Load raw data into the managed session index.
2. **Examine**: Inventory the cohort and equipment.
3. **Configure**: Define privacy tags and redaction rules.
4. **Audit (Target)**: Measure PHI risks against the configuration.
5. **Backup**: (Optional) Securely lock original identities for reversibility.
6. **Anonymize**: Apply remediation to metadata (in-memory).
7. **Redact**: Scrub pixel data for specific machines (in-memory).
8. **Verify**: Re-audit the session to ensure a clean state.
9. **Report**: Generate a signed Compliance Report (Manifest, Exceptions, Audit Trail).
10. **Export**: Write clean DICOM files to disk.

In [ ]:
%pip install "git+https://github.com/kvnlng/Gantry.git"

### Initialize a Session

Gantry uses a **persistent session** to manage your workflow. Unlike scripts that run once and forget, a Session creates a local SQLite database (`gantry.db`) to index your data. This allows you to pause, resume, and audit your work without re-scanning thousands of files.

In [1]:
from gantry import Session
session = Session("cohort.db")

Initializing new session at cohort.db...


### Ingest & Examine

Ingestion builds a lightweight **metadata index** of your DICOM files. Gantry scans your folders recursively, extracting patient/study/series information into the database *without moving or modifying your original files*. It is resilient to nested directories and non-DICOM clutter.

In [4]:
session.ingest("comprehensive_dicoms")
session.save() # Persist the index to disk

Ingesting from 'comprehensive_dicoms'...


Ingesting: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:00<00:00, 587.25it/s]

ERROR: Import Failed: Missing SOPInstanceUID. Likely not a valid DICOM file.
ERROR: Import Failed: Missing SOPInstanceUID. Likely not a valid DICOM file.
ERROR: Import Failed: Missing SOPInstanceUID. Likely not a valid DICOM file.
ERROR: Import Failed: Missing SOPInstanceUID. Likely not a valid DICOM file.
ERROR: Import Failed: Missing SOPInstanceUID. Likely not a valid DICOM file.

Ingestion Complete.
Summary:
  - 135 Patients
  - 135 Studies
  - 135 Series
  - 135 Instances
Remember to call .save() to persist changes.



In [5]:
# Print a summary of the cohort and equipment
session.examine()


Inventory Summary:
 Patients:  135
 Studies:   135
 Series:    135
 Instances: 135

Equipment Inventory:
 - Agfa - CR 30-X (Count: 5)
 - Agfa - DX-D (Count: 5)
 - Carestream - Classic (Count: 5)
 - Carestream - DRX (Count: 5)
 - Fuji - FCR (Count: 5)
 - Fuji - FDR (Count: 5)
 - GE - Discovery (Count: 5)
 - GE - Discovery MI (Count: 5)
 - GE - Innova (Count: 5)
 - GE - NM 830 (Count: 5)
 - GE - Precision (Count: 5)
 - GE - Revolution (Count: 5)
 - GE - Senographe (Count: 5)
 - GE - Voluson (Count: 5)
 - Gantry - Test (Count: 5)
 - Generic - ScreenCapture (Count: 5)
 - Hologic - Selenia (Count: 5)
 - Philips - Brilliance (Count: 5)
 - Philips - EPIQ (Count: 5)
 - Philips - Ingenia (Count: 5)
 - Siemens - Acuson (Count: 5)
 - Siemens - Artis (Count: 5)
 - Siemens - Biograph (Count: 5)
 - Siemens - Luminos (Count: 5)
 - Siemens - Magnetom (Count: 5)
 - Siemens - Somatom (Count: 5)
 - Siemens - Symbia (Count: 5)


### Configure & Audit

Before changing anything, define your privacy rules. 

Use `create_config` to generate a scaffolding based on your inventory, then `audit` to scan that inventory against your rules. 

In [7]:
# Create a default configuration file
session.create_config("config.yaml")

# Load the configuration (rules, tags, jitter)
session.load_config("config.yaml")

Scaffolded Unified Config to config.yaml
Loading configuration from config.yaml...
Configuration Loaded:
 - 135 Machine Redaction Rules
 - 29 PHI Tags
 - Date Jitter: -365 to -1 days
 - Remove Private Tags: True
Tip: Run .audit() to check PHI, or .redact_pixels() to apply redaction.


In [8]:
# Run an audit to find PHI
report = session.audit() 
session.save_analysis(report)

print(f"Found {len(report)} potential PHI issues.")

Scanning PHI: 100%|███████████████████████████████████████████████████████████████████████████████████| 135/135 [00:00<00:00, 218453.33it/s]

Found 1890 potential PHI issues.


### Backup Identity (Optional)

To enable reversible anonymization, generate a cryptographic key and "lock" the original patient identities into a secure, encrypted DICOM tag. This must be done *before* anonymization.

In [9]:
# Enable encryption (generates 'gantry.key')
session.enable_reversible_anonymization()

# cryptographically lock identities for all patients found in the audit
# Optional: Specify custom tags to preserve (defaults to Name, ID, DOB, Sex, Accession)
# session.lock_identities(report, tags_to_lock=["0010,0010", "0010,0020", "0010,0030"])
session.lock_identities(report)

session.save()

Locking Identities: 100%|█████████████████████████████████████████████████████████████████████████| 135/135 [00:00<00:00, 11000.54patient/s]


Gantry supports **Exploratory Data Analysis (EDA)**. You can interrogate your cohort using Pandas and perform targeted exports based on metadata criteria.

In [10]:
df = session.export_dataframe(expand_metadata=True)
df.loc[df.Modality == 'CT', ['PatientID', 'PatientName']][0:10]

<frozen importlib._bootstrap>:491: RuntimeWarning: The global interpreter lock (GIL) has been enabled to load module 'pandas._libs.pandas_parser', which has not declared that it can run safely without the GIL. To override this behavior and keep the GIL disabled (at your own risk), run with PYTHON_GIL=0 or -Xgil=0.


,PatientID,PatientName
13,PID-92766,Jones^Joseph
21,PID-81096,Wilson^Barbara
29,PID-25596,Johnson^John
36,PID-46626,Miller^Susan
49,PID-51981,Gonzalez^Jennifer
54,PID-28544,Hernandez^Linda
60,PID-94275,Gonzalez^Linda
79,PID-30308,Davis^Mary
81,PID-37839,Martinez^John
97,PID-83322,Rodriguez^Robert


**Anonymize**: Strips or replaces metadata tags (PatientID, Names, Dates) based on your config.

In [11]:
# Apply metadata remediation (anonymization) using the findings
session.anonymize(report)
session.save()

Anonymizing Metadata: 100%|█████████████████████████████████████████████████████████████████████| 1890/1890 [00:00<00:00, 40956.83finding/s]

Stamping De-Identification Method tags...


In [12]:
df = session.export_dataframe(expand_metadata=True)
df.loc[df.Modality == 'CT', ['PatientID', 'PatientName']][0:10]

,PatientID,PatientName
13,ANON_6f4416fb3d06,ANONYMIZED
21,ANON_aae651838799,ANONYMIZED
29,ANON_2dc5f35b2132,ANONYMIZED
36,ANON_1ef8c182fbf5,ANONYMIZED
49,ANON_acfe0fae7196,ANONYMIZED
54,ANON_d55677246438,ANONYMIZED
60,ANON_5a6302ea5a55,ANONYMIZED
79,ANON_d6094d4a01de,ANONYMIZED
81,ANON_bb90a3e2a05d,ANONYMIZED
97,ANON_30b35fd16fd5,ANONYMIZED


### Recover Identity (Optional)

If you have a valid key (`gantry.key`) and need to retrieve the original identity of an anonymized patient:

In [ ]:
def recover(anon):
    session.recover_patient_identity(anon.PatientID, restore=True)

df.apply(recover, axis=1)
session.save()

In [14]:
df = session.export_dataframe(expand_metadata=True)
df.loc[df.Modality == 'CT', ['PatientID', 'PatientName']][0:10]

,PatientID,PatientName
13,PID-92766,Jones^Joseph
21,PID-81096,Wilson^Barbara
29,PID-25596,Johnson^John
36,PID-46626,Miller^Susan
49,PID-51981,Gonzalez^Jennifer
54,PID-28544,Hernandez^Linda
60,PID-94275,Gonzalez^Linda
79,PID-30308,Davis^Mary
81,PID-37839,Martinez^John
97,PID-83322,Rodriguez^Robert


**Redact**: Loads pixel data and scrubs burned-in PHI from defined regions.

In [15]:
# Apply pixel redaction rules (requires config to be loaded)
session.redact()

No matching images found for any loaded rules.


**Export**: The final "Gatekeeper". Writes clean files to a new directory. Setting `safe=True` ensures the export halts if any verification checks fail (e.g., corrupt images or missing codecs).

In [16]:
df = session.export_dataframe(expand_metadata=True)
cohort_df = df[df.Modality == 'CT']
# Export only safe (clean) data to a new folder
# Compression="j2k" optionally compresses output to JPEG 2000
session.export("export_clean", subset=cohort_df, safe=True, compression="j2k")

Preparing for export (saving & releasing memory)...


Releasing Memory: 100%|█████████████████████████████████████████████████████████████████████████████| 135/135 [00:00<00:00, 1513986.74img/s]

Memory Cleanup: Released 135 images from RAM.
Exporting...
Running safety scan...



Scanning PHI: 100%|██████████████████████████████████████████████████████████████████████████████████████| 135/135 [00:00<00:00, 661.30it/s]

Safety Scan Found Issues: 135 Patients, 0 Studies contain PHI.
The following tags were flagged as dirty:
Tag             Description                    Count      Examples
--------------------------------------------------------------------------------
0010,0010       patient_name                   270        Lopez^Barbara, Miller^Sarah, Rodriguez^Charles
0010,0020       patient_id                     270        PID-40488, PID-78880, PID-35622
0010,0030       Patient's Birth Date           135        19920302, 19881006, 19660608
0008,0050       Accession Number               135        ACC-132125, ACC-491366, ACC-735123

To allow export, you must either REMOVE these tags or mark them as KEEP in your configuration.
Suggested Config Update:
{
    "phi_tags": {
        "0010,0010": {
            "action": "REMOVE",
            "name": "patient_name"
        },
        "0010,0020": {
            "action": "REMOVE",
            "name": "patient_id"
        },
        "0010,0030": {
        


Exporting: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 69.71it/s]

Done.


### Automated Compliance Reports

Generate single-step, audit-ready Markdown reports for HIPAA/GDPR documentation. Reports include:

- **Cohort Manifest**: Summary of all processed patients/studies.
- **Audit Trail**: Aggregated counts of every action (Anonymize, Redact, Export).
- **Exceptions**: Explicit listing of any warnings or errors encountered.
- **Validation Status**: Automatic `PASS`/`REVIEW_REQUIRED` grading.

In [17]:
# Generate a Markdown report
session.generate_report("compliance_report.md")

In [18]:
from IPython.display import Markdown, display

with open('compliance_report.md', 'r', encoding='utf-8') as f:
    content = f.read()
display(Markdown(content))

# Compliance Report

**Generated At:** 2026-01-14 20:12:47
**Project:** cohort.db
**System Version:** Gantry v0.5.3

## 1. Executive Summary

| Metric | Value |
| :--- | :--- |
| **Validation Status** | **REVIEW_REQUIRED** |
| Total Patients | 135 |
| Total Instances | 135 |
| Privacy Profile | See Config |
| De-ID Method | Safe Harbor (Basic Profile) |

## 2. Processing Audit

The following actions were recorded in the secure audit trail:

| Action Type | Count |
| :--- | :--- |
| REMEDIATION_REMOVE | 1215 |
| REMEDIATION_REPLACE | 405 |
| REMEDIATION_SHIFT_DATE | 270 |

## 3. Exceptions & Errors

> [!WARNING]
> The following issues were encountered during processing:

| Timestamp | Action | Details |
| :--- | :--- | :--- |
| 2026-01-14T20:12:47.276723 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.16023.86.1 |
| 2026-01-14T20:12:47.276743 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.18267.18.1 |
| 2026-01-14T20:12:47.276749 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.83920.41.1 |
| 2026-01-14T20:12:47.276751 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.99812.5.1 |
| 2026-01-14T20:12:47.276754 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.86731.73.1 |
| 2026-01-14T20:12:47.276757 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.69402.87.1 |
| 2026-01-14T20:12:47.276759 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.46108.57.1 |
| 2026-01-14T20:12:47.276762 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.53292.93.1 |
| 2026-01-14T20:12:47.276764 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.12153.75.1 |
| 2026-01-14T20:12:47.276767 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.46564.31.1 |
| 2026-01-14T20:12:47.276769 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.27002.42.1 |
| 2026-01-14T20:12:47.276771 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.57553.12.1 |
| 2026-01-14T20:12:47.276774 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.37362.35.1 |
| 2026-01-14T20:12:47.276776 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.22976.48.1 |
| 2026-01-14T20:12:47.276778 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.79646.83.1 |
| 2026-01-14T20:12:47.276780 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.77509.70.1 |
| 2026-01-14T20:12:47.276783 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.55201.86.1 |
| 2026-01-14T20:12:47.276786 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.24092.29.1 |
| 2026-01-14T20:12:47.276788 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.52181.82.1 |
| 2026-01-14T20:12:47.276798 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.15606.33.1 |
| 2026-01-14T20:12:47.276800 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.77089.80.1 |
| 2026-01-14T20:12:47.276808 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.50468.4.1 |
| 2026-01-14T20:12:47.276813 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.99079.97.1 |
| 2026-01-14T20:12:47.276816 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.32983.32.1 |
| 2026-01-14T20:12:47.276819 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.83292.22.1 |
| 2026-01-14T20:12:47.276821 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.57323.24.1 |
| 2026-01-14T20:12:47.276824 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.85453.8.1 |
| 2026-01-14T20:12:47.276827 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.46012.14.1 |
| 2026-01-14T20:12:47.276829 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.50402.58.1 |
| 2026-01-14T20:12:47.276832 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.12141.82.1 |
| 2026-01-14T20:12:47.276834 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.13385.62.1 |
| 2026-01-14T20:12:47.276837 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.60374.18.1 |
| 2026-01-14T20:12:47.276839 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.35388.3.1 |
| 2026-01-14T20:12:47.276842 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.44928.27.1 |
| 2026-01-14T20:12:47.276845 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.91782.80.1 |
| 2026-01-14T20:12:47.276847 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.63089.19.1 |
| 2026-01-14T20:12:47.276849 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.95794.88.1 |
| 2026-01-14T20:12:47.276852 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.66468.3.1 |
| 2026-01-14T20:12:47.276854 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.32597.94.1 |
| 2026-01-14T20:12:47.276856 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.97971.88.1 |
| 2026-01-14T20:12:47.276859 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.92925.41.1 |
| 2026-01-14T20:12:47.276864 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.12968.2.1 |
| 2026-01-14T20:12:47.276869 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.75835.7.1 |
| 2026-01-14T20:12:47.276873 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.84152.65.1 |
| 2026-01-14T20:12:47.276878 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.53469.33.1 |
| 2026-01-14T20:12:47.276882 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.33062.6.1 |
| 2026-01-14T20:12:47.276884 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.77706.76.1 |
| 2026-01-14T20:12:47.276890 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.94049.9.1 |
| 2026-01-14T20:12:47.276894 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.79614.3.1 |
| 2026-01-14T20:12:47.276897 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.70762.62.1 |
| 2026-01-14T20:12:47.276899 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.87877.45.1 |
| 2026-01-14T20:12:47.276907 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.53441.34.1 |
| 2026-01-14T20:12:47.276910 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.62489.53.1 |
| 2026-01-14T20:12:47.276913 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.80576.73.1 |
| 2026-01-14T20:12:47.276915 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.71640.76.1 |
| 2026-01-14T20:12:47.276918 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.97379.10.1 |
| 2026-01-14T20:12:47.276923 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.61906.2.1 |
| 2026-01-14T20:12:47.276925 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.79596.90.1 |
| 2026-01-14T20:12:47.276930 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.31921.25.1 |
| 2026-01-14T20:12:47.276933 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.87248.90.1 |
| 2026-01-14T20:12:47.276936 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.97795.18.1 |
| 2026-01-14T20:12:47.276938 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.12701.66.1 |
| 2026-01-14T20:12:47.276946 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.39754.18.1 |
| 2026-01-14T20:12:47.276949 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.51513.27.1 |
| 2026-01-14T20:12:47.276952 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.28932.54.1 |
| 2026-01-14T20:12:47.276954 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.69565.1.1 |
| 2026-01-14T20:12:47.276957 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.81351.40.1 |
| 2026-01-14T20:12:47.276960 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.97267.72.1 |
| 2026-01-14T20:12:47.276963 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.50170.24.1 |
| 2026-01-14T20:12:47.276965 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.21587.70.1 |
| 2026-01-14T20:12:47.276968 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.75382.68.1 |
| 2026-01-14T20:12:47.276971 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.81658.93.1 |
| 2026-01-14T20:12:47.276974 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.55093.45.1 |
| 2026-01-14T20:12:47.276977 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.59763.68.1 |
| 2026-01-14T20:12:47.276982 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.34711.32.1 |
| 2026-01-14T20:12:47.276986 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.75353.36.1 |
| 2026-01-14T20:12:47.276989 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.75385.72.1 |
| 2026-01-14T20:12:47.276992 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.10851.5.1 |
| 2026-01-14T20:12:47.276994 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.69726.8.1 |
| 2026-01-14T20:12:47.276997 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.41389.48.1 |
| 2026-01-14T20:12:47.277000 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.73655.11.1 |
| 2026-01-14T20:12:47.277003 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.71136.69.1 |
| 2026-01-14T20:12:47.277006 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.36892.14.1 |
| 2026-01-14T20:12:47.277008 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.42986.43.1 |
| 2026-01-14T20:12:47.277011 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.24900.7.1 |
| 2026-01-14T20:12:47.277014 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.53247.57.1 |
| 2026-01-14T20:12:47.277017 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.24246.42.1 |
| 2026-01-14T20:12:47.277019 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.42538.31.1 |
| 2026-01-14T20:12:47.277022 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.18787.50.1 |
| 2026-01-14T20:12:47.277024 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.63032.59.1 |
| 2026-01-14T20:12:47.277026 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.76312.72.1 |
| 2026-01-14T20:12:47.277029 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.55495.42.1 |
| 2026-01-14T20:12:47.277031 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.15385.52.1 |
| 2026-01-14T20:12:47.277034 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.15813.6.1 |
| 2026-01-14T20:12:47.277036 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.24635.29.1 |
| 2026-01-14T20:12:47.277039 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.69400.95.1 |
| 2026-01-14T20:12:47.277041 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.30072.34.1 |
| 2026-01-14T20:12:47.277044 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.98596.95.1 |
| 2026-01-14T20:12:47.277046 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.22402.68.1 |
| 2026-01-14T20:12:47.277048 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.81418.6.1 |
| 2026-01-14T20:12:47.277051 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.96516.94.1 |
| 2026-01-14T20:12:47.277053 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.58123.77.1 |
| 2026-01-14T20:12:47.277056 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.40209.50.1 |
| 2026-01-14T20:12:47.277058 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.28569.61.1 |
| 2026-01-14T20:12:47.277061 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.76536.55.1 |
| 2026-01-14T20:12:47.277064 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.55427.85.1 |
| 2026-01-14T20:12:47.277066 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.15924.7.1 |
| 2026-01-14T20:12:47.277069 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.83845.87.1 |
| 2026-01-14T20:12:47.277072 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.60201.6.1 |
| 2026-01-14T20:12:47.277075 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.59786.57.1 |
| 2026-01-14T20:12:47.277077 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.93078.99.1 |
| 2026-01-14T20:12:47.277080 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.89746.10.1 |
| 2026-01-14T20:12:47.277082 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.50841.91.1 |
| 2026-01-14T20:12:47.277085 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.31142.84.1 |
| 2026-01-14T20:12:47.277087 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.52112.41.1 |
| 2026-01-14T20:12:47.277089 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.35622.59.1 |
| 2026-01-14T20:12:47.277092 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.10096.78.1 |
| 2026-01-14T20:12:47.277095 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.54758.43.1 |
| 2026-01-14T20:12:47.277097 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.29587.44.1 |
| 2026-01-14T20:12:47.277100 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.63787.69.1 |
| 2026-01-14T20:12:47.277103 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.66948.27.1 |
| 2026-01-14T20:12:47.277105 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.54719.71.1 |
| 2026-01-14T20:12:47.277107 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.44263.8.1 |
| 2026-01-14T20:12:47.277110 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.89213.53.1 |
| 2026-01-14T20:12:47.277112 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.69116.95.1 |
| 2026-01-14T20:12:47.277114 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.34374.43.1 |
| 2026-01-14T20:12:47.277117 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.80674.7.1 |
| 2026-01-14T20:12:47.277119 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.13378.7.1 |
| 2026-01-14T20:12:47.277122 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.51239.44.1 |
| 2026-01-14T20:12:47.277125 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.47793.77.1 |
| 2026-01-14T20:12:47.277127 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.36903.14.1 |
| 2026-01-14T20:12:47.277129 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.66665.65.1 |
| 2026-01-14T20:12:47.277132 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.96662.37.1 |
| 2026-01-14T20:12:47.277134 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.79811.48.1 |
| 2026-01-14T20:12:47.277136 | COMPLIANCE_CHECK | BurnedInAnnotation FLAGGED as YES - 1.2.840.111111.44750.98.1 |

## 4. Cohort Manifest

| Study UID | Patient ID | Instance Count |
| :--- | :--- | :--- |
| 1.2.840.111111.16023 | PID-40488 | 1 |
| 1.2.840.111111.18267 | PID-35622 | 1 |
| 1.2.840.111111.83920 | PID-78880 | 1 |
| 1.2.840.111111.99812 | PID-98402 | 1 |
| 1.2.840.111111.86731 | PID-11042 | 1 |
| 1.2.840.111111.69402 | PID-87846 | 1 |
| 1.2.840.111111.46108 | PID-60017 | 1 |
| 1.2.840.111111.53292 | PID-70615 | 1 |
| 1.2.840.111111.12153 | PID-45733 | 1 |
| 1.2.840.111111.46564 | PID-65539 | 1 |

*...and 125 more studies.*

## 5. Validation & Verification

*   **Identified Issues:** 0
*   **Methodology:** The dataset was processed using the Gantry Safe Harbor pipeline. Pixel data was scanned against machine-specific redaction zones. Metadata was remediated according to DICOM PS3.15 See Config profile.
*   **Verification Details:** Standard automated checks performed.

---
**Data Protection Officer Signature:**

__________________________________________________
*(Date)*
